In [2]:
# dataframe management
import pandas as pd
import math
import numpy as np
import random
import sklearn
import time
import json
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator
from pyomo.environ import *
from pyomo.opt import SolverFactory
from source.util import *
from source.maths1 import *
from source.sorct import *
from sklearn.model_selection import KFold

In [3]:
data = pd.read_csv('seeds_data.csv',sep=";")
data_std = data.copy()
data.head(5)

,A,P,C,len_kernel,width_kernel,asymm_coef,len_ker_groove,Classes
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [4]:
# Setting of the Scaling (to interval (0,1)) phase
scaler = MinMaxScaler()

#Preprocessing: we get the columns names of features which have to be standardized
columns_names = list(data)
index_features = list(range(0,len(data_std.columns)-1))

#The name of the classes K
classes = data_std['Classes'].unique().tolist()
classes_en = [i for i in range(len(classes))] 

#Encoder processing
le = preprocessing.LabelEncoder()
le.fit(data_std['Classes'])

data_std['Classes'] = le.transform(data_std['Classes']) 

#Scaling phase
data_std[columns_names[0:len(index_features)]] = scaler.fit_transform(data_std[columns_names[0:len(index_features)]])
data_std = data_std.replace(1.0000000000000002, 1)
data_std = data_std.replace(1.0000000000000004, 1)
data_std = data_std.replace(1.0000000000000009, 1)

In [5]:
data_std = data_std.sample(frac= 1 ,random_state = 655).reset_index(drop=True)

In [6]:
init = {'init_data': [1324, 4162, 2354, 8395, 2009, 770, 166, 281, 6181, 901, 405],
 'init_var': [[8838, 8390, 6503, 1145, 2352, 5128, 5022, 8365, 427],
  [2569, 8549, 9216, 8087, 5029, 9208, 2576, 6217, 75],
  [517, 230, 246, 980, 8429, 59, 617, 8361, 2983],
  [4895, 2471, 3107, 3044, 1163, 1173, 9186, 2324, 1278],
  [8236, 2191, 9127, 3335, 5876, 7880, 6096, 3163, 649],
  [2784, 5850, 2161, 512, 758, 1737, 829, 3741, 943],
  [7951, 8064, 6754, 2009, 6507, 8895, 8320, 7779, 792],
  [4138, 5831, 5883, 9000, 6780, 9391, 8144, 9610, 109],
  [486, 21, 776, 958, 557, 494, 319, 12, 3452],
  [7863, 2522, 6532, 7737, 7316, 5823, 3838, 2940, 9528]]}

In [7]:
# DICTIONARY FOR ACCURACIES DELTAS AND TIMING
acc_s ={'s':[]}
delta_g ={'s':[]}
delta_l ={'s':[]}

In [8]:
k_fold = KFold(n_splits=5)
lambdas = 1/(3*(len(data_std.columns)-1))
l = 1/(len(data_std.columns)-1)
i = 0
for train_indices, test_indices in k_fold.split(data_std):
    
    df_train = data_std.iloc[train_indices]
    X_test = data_std.iloc[test_indices,0:data_std.shape[1]-1]
    y_test = data_std.iloc[test_indices,data_std.shape[1]-1:data_std.shape[1]]
    I_in_k = inst_class(df_train)
    def I_k(model,i):
        if i==0:
            return I_in_k[0]
        elif i==1:
            return I_in_k[1]
        elif i==2:
            return I_in_k[2]
    
    # INITIALIZATION FOR VARIABLES
    np.random.seed(init['init_var'][i][0])
    init_a = np.random.uniform(low=-1.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][1])
    init_mu = np.random.uniform(low=-1.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][2])
    init_C = np.random.uniform(low=0.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][3])
    init_P = np.random.uniform(low=0.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][4])
    init_p = np.random.uniform(low=0.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][5])
    init_beta = np.random.uniform(low=0.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][6])
    init_am = np.random.uniform(low=0.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][7])
    init_ap = np.random.uniform(low=0.0, high=1.0, size=None)
    np.random.seed(init['init_var'][i][8])
    init_z = np.random.uniform(low=0.0, high=1.0, size=None)
    i += 1
    
    ini = [init_a,init_mu,init_C,init_P,init_p,init_am,init_ap,init_beta,init_z]
    
    # simple
    simple = SORCT(df_train,I_in_k,I_k)
    simple.set_init(ini)
    simple.createModel()
    simple.set_alpha(5)
    simple.charge_of("simple",0)
    
    # SOLVING PHASE: twice to be sure to reach the solution
    # SIMPLE
    print("SIMPLE")
    try:
        simple.solve()
    except ValueError:
        print ('Invalid value!')
    try:
        simple.solve()
    except ValueError:
        print ('Invalid value!')
    
    
    
    simple.extraction_va()

    # PREDICTION PHASE
    
    p_s = simple.predicted_lab(X_test)
    acc_s['s'].append(simple.accuracy(p_s,y_test.values))
    delta_l['s'].append(simple.delta_l())
    delta_g['s'].append(simple.delta_g())

    

Init values defined.
Objective function loaded
SIMPLE
Init values defined.
Objective function loaded
SIMPLE
Init values defined.
Objective function loaded
SIMPLE
Init values defined.
Objective function loaded
SIMPLE
Init values defined.
Objective function loaded
SIMPLE


In [10]:
acc_s

{'s': [0.9285714285714286,
  0.9047619047619048,
  0.9761904761904762,
  1.0,
  0.9523809523809523]}

In [11]:
delta_l

{'s': [66.6667, 66.6667, 66.6667, 100.0, 100.0]}

In [13]:
delta_g

{'s': [100.0, 100.0, 100.0, 100.0, 100.0]}